<img src="http://codigodelsur.com/wp-content/uploads/2017/11/1_IvCDlfi3vQfgyKO1eFv4jA.png" 
alt="IMAGE ALT TEXT HERE" width="240" height="180" /></img>


## Spokane Python  --  Using GraphQL in Python


The goal of this talk is to help people understand what GraphQL is and why it's useful. I will also discuss best practices when using GraphQL and how to use GraphQL in Python.




#### First, in order to better understand the need for GraphQL we first need to look at the structure of a simple REST API. Let's use the Star Wars API (SWAPI).

You could even go back further and look at the [landscape before REST](https://blog.readme.io/the-history-of-rest-apis/). Remember SOAP? Did you ever use CORBA?

### Let's use the [Star Wars REST API](https://swapi.co/) as an example

In [ ]:
import pprint
import requests

# Query Luke Skywalker
resp = requests.get('https://swapi.co/api/people/1/')
pprint.pprint(resp.json())

Above we are doing a simple REST request for __/people/1/__

### How do we load Luke's films and star ships?

In [ ]:
resp = requests.get('https://swapi.co/api/people/1/')

# As you can see this requires multiple round trips to the REST API server in order to get the data I need
lukes_films = [requests.get(film_url).json() for film_url in resp.json().get('films', [])]
lukes_starships = [requests.get(starship_url).json() for starship_url in resp.json().get('starships', [])]

print("Luke's Films...\n")
pprint.pprint(lukes_films)

print("\nLuke's Star Ships...\n")
pprint.pprint(lukes_starships)

### Is there a better/more efficient way to access the data that I need through REST? Maybe HTTP/2 Multiplexing?

Without creating an explosion of custom REST endoints, there really isn't a better way to achieve these kind of queries via REST. Designing a RESTful API requires careful coordination between the backend developers and the frontend developers.

### GraphQL to the Rescue

![alt text](http://guimperarnau.com/files/blog/Fantastic-GANs-and-where-to-find-them/hype_train.jpg "Credit: ")


#### So what is GraphQL?
> GraphQL is a query language for your API. GraphQL isn't tied to any specific database or storage engine and is instead backed by your existing code and data.
> -- [graphql.org](http://graphql.org/learn/)

##### Awesome Resources

* https://graphql.org
* https://www.howtographql.com/
* https://dev-blog.apollodata.com/
* https://github.com/chentsulin/awesome-graphql

Yes calling it GraphQL is confusing.

Here GraphQL example that simplifies the above REST API queries...

[Example Star Wars API in GraphQL](http://graphql.org/swapi-graphql/)

Please try to look past the semantics of "*Connection", "edges" and "node". These are [Relay semantics](https://facebook.github.io/relay/graphql/connections.htm).

```bash
{
  person(id: "cGVvcGxlOjE=") {
    name
    birthYear
    eyeColor
    films {
      id
      title
    }
    starships {
      id
      name
      model
    }
  }
}
```

### GraphQL's Killer App GraphiQL

Do people really keep up with docs? Nope. Ugh, Swagger docs.



* https://developer.github.com/v4/explorer/   (Yelp uses Graphene!)
* https://www.yelp.com/developers/graphiql

### [Python Graphene](http://graphene-python.org/)

In [ ]:
# GraphQL is not bound to a protocol, yet people 

import json
import graphene



class Query(graphene.ObjectType):
    hello = graphene.String()
    foo = graphene.String()
    baz = graphene.Int()
    
    def resolve_hello(self, info):
        return 'World'

    def resolve_foo(self, info):
        return 'Bar'

    

schema = graphene.Schema(query=Query)

resp = schema.execute('''
  query {
    hello
    foo
  }
''')

print(json.dumps(resp.data, indent=4))



### More Complex Example

In [ ]:
# React's Jest framework has popularized Snapshot testing
# https://facebook.github.io/jest/docs/en/snapshot-testing.html
# You should look at Snapshot testing in Python using https://github.com/syrusakbary/snapshottest
import json
import unittest

unittest.TestCase.maxDiff = None

import graphene


class Gender(graphene.Enum):
    MALE = 1
    FEMALE = 2
    NA = 3


class Person(graphene.ObjectType):
    id = graphene.ID()
    name = graphene.String()
    birth_year = graphene.String()
    gender = graphene.Field(Gender)


class Query(graphene.ObjectType):
    person = graphene.Field(Person, id=graphene.ID(required=True))
    
    def resolve_person(self, info, id):
        # TODO: Implement Function
        pass
    
    
class CreatePerson(graphene.Mutation):
    class Arguments:
        id = graphene.ID()
        name = graphene.String()
        birth_year = graphene.String()
        gender = graphene.Argument(Gender, default_value=Gender.NA.value)

    ok = graphene.Boolean()
    person = graphene.Field(Person)

    def mutate(self, info, name, birth_year, gender):
        # TODO: Implement Function
        pass


class Mutation(graphene.ObjectType):
    create_person = CreatePerson.Field()


schema = graphene.Schema(query=Query, mutation=Mutation)


class TestGrapheneQuery(unittest.TestCase):
    
    def assertResponseEqual(self, resp, expected):
        self.assertIsNone(resp.errors)
        self.assertEquals(json.dumps(resp.data, indent=4), json.dumps(expected, indent=4))
        
    def test_retrieve_person(self):
        resp = schema.execute('''
          query {
            person(id: "1"){
                id
                name
                birthYear
                gender
            }
          }
        ''')
        expected = {
            "person": {
                "id": "1",
                "name": "Luke Skywalker",
                "birthYear": "19BBY",
                "gender": "MALE"
             }
        }
        self.assertResponseEqual(resp, expected)
        
    def test_create_person(self):
        resp = schema.execute('''
          mutation {
            createPerson(name: "C-3PO", birthYear: "112BBY", gender: NA){
                person{
                    id
                    name
                    birthYear
                    gender
                }
                ok
            }
          }
        ''')
        expected = {
            "createPerson": {
                "person": {
                    "id": "2",
                    "name": "C-3PO",
                    "birthYear": "112BBY",
                    "gender": "NA"
                },
                "ok": True
             }
        }
        self.assertResponseEqual(resp, expected)
        

suite = unittest.TestLoader().loadTestsFromModule(TestGrapheneQuery())
unittest.TextTestRunner().run(suite)

### NCAA March Madness 2018

https://en.wikipedia.org/wiki/March_Madness_pools

In [ ]:
# Let's scrape some data!
import requests

resp = requests.get('https://www.cbssports.com/data/collegebasketball/ncaa-tournament/brackets/update/2017/NCAAB?as=json')
tournament = resp.json().get('tournament')
bracket = tournament.get('bracket')
locations = bracket.get('locations')
regions = bracket.get('regions').get('region')
rounds_by_region = [round.get('rounds').get('round') for round in regions]
teams = bracket.get('teams', {}).get('team')


print(json.dumps(teams, indent=4))


In [ ]:
# let's add some data to redis and elasticsearch!
import arrow
import unittest
import mock
import copy

unittest.TestCase.maxDiff = None


from requests_html import HTMLSession
from elasticsearch import Elasticsearch
es = Elasticsearch(['es'])
import redis
r = redis.StrictRedis(host='redis', port=6379, db=0)


class MarchMadnessDataImporter(object):
    def __init__(self, r, es):
        """Writes rows to Redis and ElasticSearch.
        
        Args:
            r: A redis instance
            es: A elasticsearch instance
        """
        self.r = r
        self.es = es
        
    def __retrieve_roster(self, abbrev):
        session = HTMLSession()
        resp = session.get('https://www.cbssports.com/collegebasketball/teams/roster/{0}'.format(abbrev))
        table = resp.html.find('.data')

        for elem in table[1:-1]:
            player_rows = [row.text for row in elem.find('tr')[3:]]
            for num, name, position, height, weight, univ_class, hometown in [player_row.split('\n') for player_row in player_rows]:
                yield {
                    'abbrev': abbrev,
                    'num': num,
                    'name': name,
                    'position': position,
                    'height': height,
                    'class': univ_class,
                    'hometown': hometown
                }

        
    def import_teams(self, teams):
        total_teams = len(teams)
        print("Preparing to import teams...")
        for team_idx, team in enumerate(teams):
            key = 'team:{0}'.format(team.get('abbrev'))
            self.r.set(key, json.dumps(team))
            self.es.index(index='teams', doc_type='team', id=key, body=team)
            for idx, player in enumerate(self.__retrieve_roster(team.get('abbrev'))):
                key = 'player:{0}:{1}'.format(team.get('abbrev'), idx)
                self.r.set(key, json.dumps(player))
                self.es.index(index='players', doc_type='player', id=key, body=player)
            print("Importing teams {0}/{1}...".format(team_idx+1, total_teams))
            
    def __get_games_by_round(self, round_num):
        south, west, east, midwest = rounds_by_region[1:-1]
        games = []
        games.extend(south[round_num].get('games').get('game'))
        games.extend(west[round_num].get('games').get('game'))
        games.extend(east[round_num].get('games').get('game'))
        games.extend(midwest[round_num].get('games').get('game'))
        return games

    def import_rounds_by_regions(self, rounds_by_region):
        all_games = []
        round_1 = self.__get_games_by_round(0)
        round_2 = self.__get_games_by_round(1)
        sweet_16 = self.__get_games_by_round(2)
        elite_8 = self.__get_games_by_round(3)
        final_4 = rounds_by_region[-1][0].get('games').get('game')
        championship = rounds_by_region[-1][1].get('games').get('game')
        all_games.extend(round_1 + round_2 + sweet_16 + elite_8 + final_4 + championship)
        for idx, game in enumerate(all_games):
            if game.get('game_abbrev'):
                away, home = game.get('game_abbrev').split('_')[2].split('@')
                game.get('game_participant')[0]['abbrev'] = home 
                game.get('game_participant')[1]['abbrev'] = away
            key = 'game:{0}'.format(idx)
            self.r.set(key, json.dumps(game))
            self.es.index(index='games', doc_type='game', id=key, body=game)

            
class TestMarchMadnessImporter(unittest.TestCase):
    
    def test_import_teams(self):
        r = mock.Mock()
        es = mock.Mock()
        importer = MarchMadnessDataImporter(r, es)
        importer.import_teams([{'abbrev': 'Gonz', 'name': 'Gonzaga', 'seed': 4}])
        r.set.assert_called_with('team:Gonz', '{"abbrev": "Gonz", "name": "Gonzaga", "seed": 4}')
        es.index.assert_called_with(index='teams', doc_type='team', id='team:Gonz', body={"abbrev": "Gonz", "name": "Gonzaga", "seed": 4})
        
    def test_import_rounds_by_regions(self):
        pass
        #TODO: implement test
    
    
suite = unittest.TestLoader().loadTestsFromModule(TestMarchMadnessImporter())
unittest.TextTestRunner().run(suite)


In [ ]:
importer = MarchMadnessDataImporter(r, es)
importer.import_rounds_by_regions(rounds_by_region)
importer.import_teams(teams)


In [ ]:
res = es.search(index="games", body={
    "query": {
        "query_string" : {
            "default_field" : "name",
            "query" : "*d*"
        }
    }
})
print(json.dumps(res, indent=4))


### Let's bulid the GraphQL schema  http://localhost

Using gunicorn and gevent, let's build a GraphQL server based on green threads.

Look at demo/application.py, demo/gunicorn_config.py and demo/schema.py

### Dataloader Pattern

Helps prevent recursive lookups sending a deluge of queries to your backend plus solves the [N+1 problem with ORMs](https://use-the-index-luke.com/sql/join/nested-loops-join-n1-problem).

* https://github.com/facebook/dataloader
* http://docs.graphene-python.org/en/latest/execution/dataloader/

In [ ]:
# %load /code/demo/loaders.py

from promise import Promise
from promise.dataloader import DataLoader
import redis

r = redis.StrictRedis(host='redis', port=6379, db=0)

class PlayerLoader(DataLoader):

    def get_player(self, obj):
        pass

    def batch_load_fn(self, keys):
        p = r.pipeline()
        for key in keys:
            p.hgetall(key)

        return Promise.resolve([get_player(key, obj) for obj in p.execute()])

### Advanced Topics
* https://dev-blog.apollodata.com/graphql-at-facebook-by-dan-schafer-38d65ef075af
* https://github.com/facebook/graphql/pull/267  GraphQL Subscriptions
